In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.6))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = 0.840845057,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


In [7]:
suffix = 'sim06.csv'
n_samples = 500

### BERT-based-uncased

In [8]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8105065666041276
RF 0.7317073170731707
KNN 0.6801125703564728
SVC 0.8030018761726079


In [10]:
# custom model wrapper for BERT
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [11]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [12]:
runAttack(bert_model_wrapper_lr_mr, n_samples, 'attack_results_new/bertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklrsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                         | 5/500 [00:58<1:36:09, 11.65s/it]textattack: Saving checkpoint under "checkpoints\1720187284774.ta.chkpt" at 2024-07-05 21:48:04 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [01:06<54:42,  6.70s/it]textattack: Saving checkpoint under "checkpoints\1720187293482.ta.chkpt" at 2024-07-05 21:48:13 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [01:43<55:41,  6.89s/it]textattack: Saving checkpoint under "checkpoints\1720187329846.ta.chkpt" at 2024-07-05 21:48:49 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 374 / 12 / 99 / 485:  97%|███████████████████▍| 485/500 [18:20<00:34,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720188326660.ta.chkpt" at 2024-07-05 22:05:26 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 378 / 12 / 100 / 490:  98%|██████████████████▌| 490/500 [18:31<00:22,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720188337834.ta.chkpt" at 2024-07-05 22:05:37 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 383 / 12 / 100 / 495:  99%|██████████████████▊| 495/500 [18:41<00:11,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720188347702.ta.chkpt" at 2024-07-05 22:05:47 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 388 / 12 / 100 / 500: 100%|███████████████████| 500/500 [18:54<00:00,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720188360903.ta.chkpt" at 2024-07-05 22:06:00 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 388 / 12 / 100 / 500: 100%|███████████████████| 500/500 [18:54<00:00,  2.27s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 388   |
| Number of failed attacks:     | 12    |
| Number of skipped attacks:    | 100   |
| Original accuracy:            | 80.0% |
| Accuracy under attack:        | 2.4%  |
| Attack success rate:          | 97.0% |
| Average perturbed word %:     | 17.7% |
| Average num. words per input: | 18.4  |
| Avg num queries:              | 63.62 |
+-------------------------------+-------+


In [13]:
runAttack(bert_model_wrapper_rf_mr, n_samples, 'attack_results_new/bertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrfsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 5/500 [00:20<33:59,  4.12s/it]textattack: Saving checkpoint under "checkpoints\1720188381659.ta.chkpt" at 2024-07-05 22:06:21 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 4 / 7:   1%|▍                           | 7/500 [00:20<24:21,  2.96s/it]

[Succeeded / Failed / Skipped / Total] 4 / 1 / 5 / 10:   2%|▌                         | 10/500 [00:32<26:44,  3.27s/it]textattack: Saving checkpoint under "checkpoints\1720188393794.ta.chkpt" at 2024-07-05 22:06:33 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 1 / 6 / 15:   3%|▊                         | 15/500 [00:48<26:23,  3.26s/it]textattack: Saving checkpoint under "checkpoints\1720188410021.ta.chkpt" at 2024-07-05 22:06:50 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 322 / 26 / 137 / 485:  97%|██████████████████▍| 485/500 [16:21<00:30,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1720189342333.ta.chkpt" at 2024-07-05 22:22:22 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 326 / 26 / 138 / 490:  98%|██████████████████▌| 490/500 [16:28<00:20,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1720189349358.ta.chkpt" at 2024-07-05 22:22:29 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 326 / 26 / 139 / 491:  98%|██████████████████▋| 491/500 [16:28<00:18,  2.01s/it]

[Succeeded / Failed / Skipped / Total] 329 / 26 / 140 / 495:  99%|██████████████████▊| 495/500 [16:38<00:10,  2.02s/it]textattack: Saving checkpoint under "checkpoints\1720189359527.ta.chkpt" at 2024-07-05 22:22:39 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 333 / 26 / 141 / 500: 100%|███████████████████| 500/500 [16:44<00:00,  2.01s/it]textattack: Saving checkpoint under "checkpoints\1720189365325.ta.chkpt" at 2024-07-05 22:22:45 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 333 / 26 / 141 / 500: 100%|███████████████████| 500/500 [16:44<00:00,  2.01s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 333    |
| Number of failed attacks:     | 26     |
| Number of skipped attacks:    | 141    |
| Original accuracy:            | 71.8%  |
| Accuracy under attack:        | 5.2%   |
| Attack success rate:          | 92.76% |
| Average perturbed word %:     | 15.39% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 65.2   |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_knn_mr, n_samples, 'attack_results_new/bertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknnsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 5/500 [00:11<19:15,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1720189377114.ta.chkpt" at 2024-07-05 22:22:57 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:   1%|▎                           | 6/500 [00:11<16:09,  1.96s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:15<12:35,  1.54s/it]textattack: Saving checkpoint under "checkpoints\1720189380855.ta.chkpt" at 2024-07-05 22:23:00 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 1 / 6 / 15:   3%|▊                         | 15/500 [00:44<23:56,  2.96s/it]textattack: Saving checkpoint under "checkpoints\1720189409861.ta.chkpt" at 2024-07-05 22:23:29 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 328 / 10 / 147 / 485:  97%|██████████████████▍| 485/500 [10:48<00:20,  1.34s/it]textattack: Saving checkpoint under "checkpoints\1720190014300.ta.chkpt" at 2024-07-05 22:33:34 after 485 attacks.
[Succeeded / Failed / Skipped / Total] 328 / 10 / 148 / 486:  97%|██████████████████▍| 486/500 [10:48<00:18,  1.34s/it]

[Succeeded / Failed / Skipped / Total] 331 / 10 / 149 / 490:  98%|██████████████████▌| 490/500 [10:54<00:13,  1.34s/it]textattack: Saving checkpoint under "checkpoints\1720190019819.ta.chkpt" at 2024-07-05 22:33:39 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 331 / 10 / 150 / 491:  98%|██████████████████▋| 491/500 [10:54<00:11,  1.33s/it]

[Succeeded / Failed / Skipped / Total] 334 / 10 / 151 / 495:  99%|██████████████████▊| 495/500 [10:59<00:06,  1.33s/it]textattack: Saving checkpoint under "checkpoints\1720190025217.ta.chkpt" at 2024-07-05 22:33:45 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 337 / 10 / 153 / 500: 100%|███████████████████| 500/500 [11:04<00:00,  1.33s/it]textattack: Saving checkpoint under "checkpoints\1720190029504.ta.chkpt" at 2024-07-05 22:33:49 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 337 / 10 / 153 / 500: 100%|███████████████████| 500/500 [11:04<00:00,  1.33s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 337    |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 153    |
| Original accuracy:            | 69.4%  |
| Accuracy under attack:        | 2.0%   |
| Attack success rate:          | 97.12% |
| Average perturbed word %:     | 8.89%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 43.12  |
+-------------------------------+--------+


In [15]:
runAttack(bert_model_wrapper_svc_mr, n_samples, 'attack_results_new/bertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvcsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:19<32:58,  4.00s/it]textattack: Saving checkpoint under "checkpoints\1720190049589.ta.chkpt" at 2024-07-05 22:34:09 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▎                           | 6/500 [00:20<27:40,  3.36s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:27<22:38,  2.77s/it]textattack: Saving checkpoint under "checkpoints\1720190057340.ta.chkpt" at 2024-07-05 22:34:17 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [00:58<31:40,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1720190088387.ta.chkpt" at 2024-07-05 22:34:48 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 373 / 16 / 96 / 485:  97%|███████████████████▍| 485/500 [17:49<00:33,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1720191099012.ta.chkpt" at 2024-07-05 22:51:39 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 377 / 16 / 97 / 490:  98%|███████████████████▌| 490/500 [17:58<00:22,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1720191108038.ta.chkpt" at 2024-07-05 22:51:48 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 382 / 16 / 97 / 495:  99%|███████████████████▊| 495/500 [18:07<00:10,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1720191117069.ta.chkpt" at 2024-07-05 22:51:57 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 387 / 16 / 97 / 500: 100%|████████████████████| 500/500 [18:20<00:00,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1720191130256.ta.chkpt" at 2024-07-05 22:52:10 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 387 / 16 / 97 / 500: 100%|████████████████████| 500/500 [18:20<00:00,  2.20s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 387    |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 97     |
| Original accuracy:            | 80.6%  |
| Accuracy under attack:        | 3.2%   |
| Attack success rate:          | 96.03% |
| Average perturbed word %:     | 16.61% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 60.91  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [16]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [18]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [19]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [20]:
runAttack(disbert_model_wrapper_lr_mr, n_samples, 'attack_results_new/disbertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklrsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:17<28:30,  3.46s/it]textattack: Saving checkpoint under "checkpoints\1720191298464.ta.chkpt" at 2024-07-05 22:54:58 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:17<20:30,  2.50s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   2%|▌                         | 10/500 [00:21<17:47,  2.18s/it]textattack: Saving checkpoint under "checkpoints\1720191302972.ta.chkpt" at 2024-07-05 22:55:02 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [00:35<19:00,  2.35s/it]textattack: Saving checkpoint under "checkpoints\1720191316447.ta.chkpt" at 2024-07-05 22:55:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 371 / 12 / 102 / 485:  97%|██████████████████▍| 485/500 [09:36<00:17,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720191857993.ta.chkpt" at 2024-07-05 23:04:17 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 375 / 12 / 103 / 490:  98%|██████████████████▌| 490/500 [09:42<00:11,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720191863496.ta.chkpt" at 2024-07-05 23:04:23 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 375 / 12 / 104 / 491:  98%|██████████████████▋| 491/500 [09:42<00:10,  1.19s/it]

[Succeeded / Failed / Skipped / Total] 379 / 12 / 104 / 495:  99%|██████████████████▊| 495/500 [09:49<00:05,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720191870912.ta.chkpt" at 2024-07-05 23:04:30 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 384 / 12 / 104 / 500: 100%|███████████████████| 500/500 [10:00<00:00,  1.20s/it]textattack: Saving checkpoint under "checkpoints\1720191881581.ta.chkpt" at 2024-07-05 23:04:41 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 384 / 12 / 104 / 500: 100%|███████████████████| 500/500 [10:00<00:00,  1.20s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 384    |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 104    |
| Original accuracy:            | 79.2%  |
| Accuracy under attack:        | 2.4%   |
| Attack success rate:          | 96.97% |
| Average perturbed word %:     | 17.18% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 61.2   |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_rf_mr, n_samples, 'attack_results_new/disbertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrfsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:15<25:07,  3.05s/it]textattack: Saving checkpoint under "checkpoints\1720191896913.ta.chkpt" at 2024-07-05 23:04:56 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 2 / 6:   1%|▎                           | 6/500 [00:15<20:58,  2.55s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   2%|▌                         | 10/500 [00:23<18:56,  2.32s/it]textattack: Saving checkpoint under "checkpoints\1720191904873.ta.chkpt" at 2024-07-05 23:05:04 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 2 / 3 / 15:   3%|▊                        | 15/500 [00:38<20:34,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1720191919878.ta.chkpt" at 2024-07-05 23:05:19 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 335 / 33 / 117 / 485:  97%|██████████████████▍| 485/500 [10:36<00:19,  1.31s/it]textattack: Saving checkpoint under "checkpoints\1720192517815.ta.chkpt" at 2024-07-05 23:15:17 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 339 / 33 / 118 / 490:  98%|██████████████████▌| 490/500 [10:45<00:13,  1.32s/it]textattack: Saving checkpoint under "checkpoints\1720192527542.ta.chkpt" at 2024-07-05 23:15:27 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 339 / 33 / 119 / 491:  98%|██████████████████▋| 491/500 [10:45<00:11,  1.32s/it]

[Succeeded / Failed / Skipped / Total] 342 / 33 / 120 / 495:  99%|██████████████████▊| 495/500 [10:49<00:06,  1.31s/it]textattack: Saving checkpoint under "checkpoints\1720192531355.ta.chkpt" at 2024-07-05 23:15:31 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 347 / 33 / 120 / 500: 100%|███████████████████| 500/500 [11:01<00:00,  1.32s/it]textattack: Saving checkpoint under "checkpoints\1720192542871.ta.chkpt" at 2024-07-05 23:15:42 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 347 / 33 / 120 / 500: 100%|███████████████████| 500/500 [11:01<00:00,  1.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 347    |
| Number of failed attacks:     | 33     |
| Number of skipped attacks:    | 120    |
| Original accuracy:            | 76.0%  |
| Accuracy under attack:        | 6.6%   |
| Attack success rate:          | 91.32% |
| Average perturbed word %:     | 17.32% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 69.14  |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_knn_mr, n_samples, 'attack_results_new/disbertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknnsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   1%|▎                           | 5/500 [00:15<26:06,  3.16s/it]textattack: Saving checkpoint under "checkpoints\1720192558768.ta.chkpt" at 2024-07-05 23:15:58 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 1 / 2 / 7:   1%|▍                           | 7/500 [00:15<18:41,  2.28s/it]

[Succeeded / Failed / Skipped / Total] 5 / 3 / 2 / 10:   2%|▌                         | 10/500 [00:26<21:28,  2.63s/it]textattack: Saving checkpoint under "checkpoints\1720192569254.ta.chkpt" at 2024-07-05 23:16:09 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 3 / 3 / 15:   3%|▊                         | 15/500 [00:33<18:00,  2.23s/it]textattack: Saving checkpoint under "checkpoints\1720192576379.ta.chkpt" at 2024-07-05 23:16:16 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 314 / 43 / 128 / 485:  97%|██████████████████▍| 485/500 [09:38<00:17,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720193121076.ta.chkpt" at 2024-07-05 23:25:21 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 318 / 43 / 129 / 490:  98%|██████████████████▌| 490/500 [09:42<00:11,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720193125451.ta.chkpt" at 2024-07-05 23:25:25 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 320 / 44 / 131 / 495:  99%|██████████████████▊| 495/500 [09:56<00:06,  1.21s/it]textattack: Saving checkpoint under "checkpoints\1720193139634.ta.chkpt" at 2024-07-05 23:25:39 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 320 / 44 / 132 / 496:  99%|██████████████████▊| 496/500 [09:56<00:04,  1.20s/it]

[Succeeded / Failed / Skipped / Total] 321 / 47 / 132 / 500: 100%|███████████████████| 500/500 [10:10<00:00,  1.22s/it]textattack: Saving checkpoint under "checkpoints\1720193153015.ta.chkpt" at 2024-07-05 23:25:53 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 321 / 47 / 132 / 500: 100%|███████████████████| 500/500 [10:10<00:00,  1.22s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 321    |
| Number of failed attacks:     | 47     |
| Number of skipped attacks:    | 132    |
| Original accuracy:            | 73.6%  |
| Accuracy under attack:        | 9.4%   |
| Attack success rate:          | 87.23% |
| Average perturbed word %:     | 11.11% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 62.75  |
+-------------------------------+--------+


In [23]:
runAttack(disbert_model_wrapper_svc_mr, n_samples, 'attack_results_new/disbertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvcsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:14<23:53,  2.90s/it]textattack: Saving checkpoint under "checkpoints\1720193167577.ta.chkpt" at 2024-07-05 23:26:07 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:14<17:08,  2.09s/it]

[Succeeded / Failed / Skipped / Total] 7 / 0 / 3 / 10:   2%|▌                         | 10/500 [00:19<16:07,  1.98s/it]textattack: Saving checkpoint under "checkpoints\1720193172852.ta.chkpt" at 2024-07-05 23:26:12 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [00:30<16:16,  2.01s/it]textattack: Saving checkpoint under "checkpoints\1720193183298.ta.chkpt" at 2024-07-05 23:26:23 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 369 / 12 / 104 / 485:  97%|██████████████████▍| 485/500 [09:50<00:18,  1.22s/it]textattack: Saving checkpoint under "checkpoints\1720193743921.ta.chkpt" at 2024-07-05 23:35:43 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 372 / 12 / 106 / 490:  98%|██████████████████▌| 490/500 [09:56<00:12,  1.22s/it]textattack: Saving checkpoint under "checkpoints\1720193749268.ta.chkpt" at 2024-07-05 23:35:49 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 372 / 12 / 107 / 491:  98%|██████████████████▋| 491/500 [09:56<00:10,  1.21s/it]

[Succeeded / Failed / Skipped / Total] 376 / 12 / 107 / 495:  99%|██████████████████▊| 495/500 [10:02<00:06,  1.22s/it]textattack: Saving checkpoint under "checkpoints\1720193755459.ta.chkpt" at 2024-07-05 23:35:55 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 381 / 12 / 107 / 500: 100%|███████████████████| 500/500 [10:13<00:00,  1.23s/it]textattack: Saving checkpoint under "checkpoints\1720193766942.ta.chkpt" at 2024-07-05 23:36:06 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 381 / 12 / 107 / 500: 100%|███████████████████| 500/500 [10:13<00:00,  1.23s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 381    |
| Number of failed attacks:     | 12     |
| Number of skipped attacks:    | 107    |
| Original accuracy:            | 78.6%  |
| Accuracy under attack:        | 2.4%   |
| Attack success rate:          | 96.95% |
| Average perturbed word %:     | 16.85% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 60.73  |
+-------------------------------+--------+


### CLIP

In [24]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [26]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [27]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [28]:
runAttack(clip_model_wrapper_lr_mr, n_samples, 'attack_results_new/clipmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklrsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:21<34:59,  4.24s/it]textattack: Saving checkpoint under "checkpoints\1720194207263.ta.chkpt" at 2024-07-05 23:43:27 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▎                           | 6/500 [00:21<29:12,  3.55s/it]

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   2%|▌                         | 10/500 [00:26<21:26,  2.63s/it]textattack: Saving checkpoint under "checkpoints\1720194212314.ta.chkpt" at 2024-07-05 23:43:32 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 0 / 1 / 15:   3%|▊                        | 15/500 [00:40<21:50,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1720194226576.ta.chkpt" at 2024-07-05 23:43:46 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 355 / 3 / 127 / 485:  97%|███████████████████▍| 485/500 [08:14<00:15,  1.02s/it]textattack: Saving checkpoint under "checkpoints\1720194680523.ta.chkpt" at 2024-07-05 23:51:20 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 358 / 3 / 129 / 490:  98%|███████████████████▌| 490/500 [08:18<00:10,  1.02s/it]textattack: Saving checkpoint under "checkpoints\1720194684943.ta.chkpt" at 2024-07-05 23:51:24 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 358 / 3 / 130 / 491:  98%|███████████████████▋| 491/500 [08:18<00:09,  1.02s/it]

[Succeeded / Failed / Skipped / Total] 361 / 3 / 131 / 495:  99%|███████████████████▊| 495/500 [08:21<00:05,  1.01s/it]textattack: Saving checkpoint under "checkpoints\1720194687577.ta.chkpt" at 2024-07-05 23:51:27 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 366 / 3 / 131 / 500: 100%|████████████████████| 500/500 [08:29<00:00,  1.02s/it]textattack: Saving checkpoint under "checkpoints\1720194695942.ta.chkpt" at 2024-07-05 23:51:35 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 366 / 3 / 131 / 500: 100%|████████████████████| 500/500 [08:29<00:00,  1.02s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 366    |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 131    |
| Original accuracy:            | 73.8%  |
| Accuracy under attack:        | 0.6%   |
| Attack success rate:          | 99.19% |
| Average perturbed word %:     | 15.24% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 52.69  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_rf_mr, n_samples, 'attack_results_new/clipmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrfsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:13<22:44,  2.76s/it]textattack: Saving checkpoint under "checkpoints\1720194709800.ta.chkpt" at 2024-07-05 23:51:49 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▍                           | 7/500 [00:13<16:19,  1.99s/it]

[Succeeded / Failed / Skipped / Total] 7 / 1 / 2 / 10:   2%|▌                         | 10/500 [00:24<20:09,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1720194720693.ta.chkpt" at 2024-07-05 23:52:00 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 1 / 3 / 15:   3%|▊                        | 15/500 [00:32<17:15,  2.13s/it]textattack: Saving checkpoint under "checkpoints\1720194728035.ta.chkpt" at 2024-07-05 23:52:08 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 318 / 23 / 144 / 485:  97%|██████████████████▍| 485/500 [09:30<00:17,  1.18s/it]textattack: Saving checkpoint under "checkpoints\1720195266203.ta.chkpt" at 2024-07-06 00:01:06 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 322 / 23 / 145 / 490:  98%|██████████████████▌| 490/500 [09:33<00:11,  1.17s/it]textattack: Saving checkpoint under "checkpoints\1720195269985.ta.chkpt" at 2024-07-06 00:01:09 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 322 / 23 / 146 / 491:  98%|██████████████████▋| 491/500 [09:34<00:10,  1.17s/it]

[Succeeded / Failed / Skipped / Total] 324 / 23 / 148 / 495:  99%|██████████████████▊| 495/500 [09:36<00:05,  1.17s/it]textattack: Saving checkpoint under "checkpoints\1720195272789.ta.chkpt" at 2024-07-06 00:01:12 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 324 / 23 / 149 / 496:  99%|██████████████████▊| 496/500 [09:36<00:04,  1.16s/it]

[Succeeded / Failed / Skipped / Total] 328 / 23 / 149 / 500: 100%|███████████████████| 500/500 [09:40<00:00,  1.16s/it]textattack: Saving checkpoint under "checkpoints\1720195277011.ta.chkpt" at 2024-07-06 00:01:17 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 328 / 23 / 149 / 500: 100%|███████████████████| 500/500 [09:41<00:00,  1.16s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 328    |
| Number of failed attacks:     | 23     |
| Number of skipped attacks:    | 149    |
| Original accuracy:            | 70.2%  |
| Accuracy under attack:        | 4.6%   |
| Attack success rate:          | 93.45% |
| Average perturbed word %:     | 13.83% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 58.84  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_knn_mr, n_samples, 'attack_results_new/clipmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknnsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   1%|▎                           | 5/500 [00:19<31:42,  3.84s/it]textattack: Saving checkpoint under "checkpoints\1720195296319.ta.chkpt" at 2024-07-06 00:01:36 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 4 / 2 / 10:   2%|▌                         | 10/500 [00:26<21:14,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1720195303113.ta.chkpt" at 2024-07-06 00:01:43 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 5 / 3 / 15:   3%|▊                         | 15/500 [00:36<19:38,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1720195313555.ta.chkpt" at 2024-07-06 00:01:53 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 288 / 45 / 152 / 485:  97%|██████████████████▍| 485/500 [09:46<00:18,  1.21s/it]textattack: Saving checkpoint under "checkpoints\1720195863221.ta.chkpt" at 2024-07-06 00:11:03 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 292 / 45 / 153 / 490:  98%|██████████████████▌| 490/500 [09:50<00:12,  1.21s/it]textattack: Saving checkpoint under "checkpoints\1720195868056.ta.chkpt" at 2024-07-06 00:11:08 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 292 / 45 / 154 / 491:  98%|██████████████████▋| 491/500 [09:51<00:10,  1.20s/it]

[Succeeded / Failed / Skipped / Total] 294 / 45 / 156 / 495:  99%|██████████████████▊| 495/500 [09:53<00:05,  1.20s/it]textattack: Saving checkpoint under "checkpoints\1720195870725.ta.chkpt" at 2024-07-06 00:11:10 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 294 / 45 / 158 / 497:  99%|██████████████████▉| 497/500 [09:53<00:03,  1.19s/it]

[Succeeded / Failed / Skipped / Total] 297 / 45 / 158 / 500: 100%|███████████████████| 500/500 [09:56<00:00,  1.19s/it]textattack: Saving checkpoint under "checkpoints\1720195873199.ta.chkpt" at 2024-07-06 00:11:13 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 297 / 45 / 158 / 500: 100%|███████████████████| 500/500 [09:56<00:00,  1.19s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 297    |
| Number of failed attacks:     | 45     |
| Number of skipped attacks:    | 158    |
| Original accuracy:            | 68.4%  |
| Accuracy under attack:        | 9.0%   |
| Attack success rate:          | 86.84% |
| Average perturbed word %:     | 10.53% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 60.15  |
+-------------------------------+--------+


In [31]:
runAttack(clip_model_wrapper_svc_mr, n_samples, 'attack_results_new/clipmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvcsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:13<22:38,  2.74s/it]textattack: Saving checkpoint under "checkpoints\1720195886996.ta.chkpt" at 2024-07-06 00:11:26 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▍                           | 7/500 [00:13<16:14,  1.98s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   2%|▌                         | 10/500 [00:18<14:45,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720195891344.ta.chkpt" at 2024-07-06 00:11:31 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   3%|▊                        | 15/500 [00:31<16:50,  2.08s/it]textattack: Saving checkpoint under "checkpoints\1720195904538.ta.chkpt" at 2024-07-06 00:11:44 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 357 / 3 / 125 / 485:  97%|███████████████████▍| 485/500 [08:26<00:15,  1.04s/it]textattack: Saving checkpoint under "checkpoints\1720196379524.ta.chkpt" at 2024-07-06 00:19:39 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 359 / 3 / 128 / 490:  98%|███████████████████▌| 490/500 [08:30<00:10,  1.04s/it]textattack: Saving checkpoint under "checkpoints\1720196383697.ta.chkpt" at 2024-07-06 00:19:43 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 363 / 3 / 129 / 495:  99%|███████████████████▊| 495/500 [08:33<00:05,  1.04s/it]textattack: Saving checkpoint under "checkpoints\1720196386962.ta.chkpt" at 2024-07-06 00:19:46 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 368 / 3 / 129 / 500: 100%|████████████████████| 500/500 [08:40<00:00,  1.04s/it]textattack: Saving checkpoint under "checkpoints\1720196393978.ta.chkpt" at 2024-07-06 00:19:53 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 368 / 3 / 129 / 500: 100%|████████████████████| 500/500 [08:40<00:00,  1.04s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 368    |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 129    |
| Original accuracy:            | 74.2%  |
| Accuracy under attack:        | 0.6%   |
| Attack success rate:          | 99.19% |
| Average perturbed word %:     | 15.01% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 51.53  |
+-------------------------------+--------+
